In [1]:
from networkx import scale_free_graph
from pymatgen.core.operations import SymmOp
from pymatgen.core.structure import Lattice, Structure
from sympy import true
from trajectory_tools.transforms import SameStructureMatcher

# Create a simple cubic structure
lattice = Lattice.cubic(3.5)
species = ["C"]
coords = [[0, 0, 0]]
original_structure = Structure(lattice, species, coords)
original_scale = 1.4

# Expand the structure into a supercell
original_structure.make_supercell([2, 2, 2])

# Rotate and translate the structure
angle = 90  # Rotation angle in degrees
axis = [0, 0, 1]  # Rotation axis (here, z-axis)
rotation_matrix = SymmOp.from_axis_angle_and_translation(axis, angle).rotation_matrix
print(f'Original rotation matrix: {rotation_matrix}')
translation_vector = [50, 0, 0]  # Translation vector
rotation_symmop = SymmOp.from_rotation_and_translation(rotation_matrix, translation_vector)

# Apply the rotation and translation to the original structure
translated_structure = original_structure.copy()
translated_structure.apply_operation(rotation_symmop)

new_volume = translated_structure.lattice.volume * original_scale
translated_structure.scale_lattice(new_volume)

# Align the transformed structure back to the original using SameStructureMatcher
matcher = SameStructureMatcher(translated_structure, original_structure, scale = True)

# Get the aligned structure
aligned_structure = matcher.get_aligned_structure()

# Get the symmetry operation used for alignment
symmop = matcher.get_symmetry_operation()
print(f'Alignment rotation matrix: {symmop.rotation_matrix}')

scale = matcher.get_volume_scale_factor()
print(f'Scale: {scale}, compared to transform scale of {1/original_scale}')

# Calculate the RMSE between the aligned structure and the original
rmse = matcher.get_rmse()
print(f"RMSE after alignment: {rmse}")


Original rotation matrix: [[ 6.123234e-17 -1.000000e+00  0.000000e+00]
 [ 1.000000e+00  6.123234e-17  0.000000e+00]
 [ 0.000000e+00  0.000000e+00  1.000000e+00]]
Alignment rotation matrix: [[ 0.00000000e+00  1.00000000e+00  4.53152255e-18]
 [-1.00000000e+00  0.00000000e+00 -4.53152255e-18]
 [-4.53152255e-18 -4.53152255e-18  1.00000000e+00]]
Scale: 0.7142857142857142, compared to transform scale of 0.7142857142857143
RMSE after alignment: 4.275472540449378e-15
